In [2]:
#Implementation of a Multi Layer Perceptrónidas
#Getting more context than the last implementation. 

In [3]:
import torch
import torch.nn.functional as F
import matplotlib as plt 
%matplotlib inline

In [4]:
#read words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
#build vocabulary of characters and mappins to integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [7]:
#build the dataset

block_size = 3 #Context lenght (numbers of characters in this case)
X, Y = [], []
for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '------->', itos[ix]) #I dont understand this line of code at all
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
C = torch.randn((27,2))
C[5]

tensor([0.4818, 0.1052])

In [9]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [10]:
W1 = torch.randn(6,100)
b1 = torch.randn(100)

In [11]:
# emb @ W1 + b1 # this does not work

In [12]:
torch.cat([emb[:,0,:],emb[:,1,:], emb[:,2,:]],1 ).shape #this obviously does not generalize
#use torch .unbind


torch.Size([228146, 6])

In [13]:
torch.cat(torch.unbind(emb, 1)) #one way to do it, efficiente#

tensor([[ 0.4954, -1.5262],
        [ 0.4954, -1.5262],
        [ 0.4954, -1.5262],
        ...,
        [-1.5325,  0.3849],
        [ 0.2484, -1.5151],
        [ 0.5356,  1.6839]])

In [14]:
a = torch.arange(18)

In [15]:
a.storage #okay this is how storage 

<bound method Tensor.storage of tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])>

In [16]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [17]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [18]:
logits = h @ W2 + b2
counts = logits.exp()

In [19]:
probs = counts / counts.sum(1, keepdims=True)

In [20]:
#-probs[torch.arange(32), Y].log().mean()

In [21]:
## Cleanup

g = torch.Generator().manual_seed(214783647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn((100), generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2,]

In [22]:
sum(p.nelement() for p in parameters)
X.shape

torch.Size([228146, 3])

In [23]:
for p in parameters:
    p.requires_grad=True

In [24]:
##fordward pass
for _ in range(100):

    ix = torch.randint(0,X.shape[0] , (32,))

    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2 # (32,27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Y[ix]) ##It's better but fuck me. 
    print(loss.item())

    
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters: 
        p. data += -.1 * p.grad

print(loss.item())

16.67683219909668
17.024560928344727
17.43882179260254
17.745935440063477
14.5660982131958
13.054605484008789
16.060863494873047
11.986722946166992
11.115150451660156
11.508943557739258
11.341324806213379
12.15728759765625
12.033713340759277
9.959010124206543
10.139669418334961
9.085213661193848
7.671845436096191
9.688244819641113
8.146636009216309
9.809962272644043
7.6501593589782715
8.782453536987305
8.75311279296875
7.812922477722168
7.11538553237915
6.263594627380371
6.223044395446777
7.440895080566406
5.24983549118042
6.976368427276611
5.859742164611816
5.415154457092285
7.111748695373535
6.223934650421143
7.135712623596191
6.3170084953308105
6.049132347106934
5.7563323974609375
5.837939739227295
5.5609564781188965
5.758042812347412
4.005717754364014
4.319870948791504
5.618676662445068
5.391717910766602
4.44871711730957
5.118844032287598
4.922367095947266
5.125599384307861
4.363006114959717
4.669479846954346
5.353888988494873
4.567012310028076
5.171993255615234
4.850686073303223
5

In [25]:
#this might not be the way

In [26]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2 # (32,27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y) ##It's better but fuck me. 
print(loss.item())

3.5278384685516357


In [27]:
## Separate this bullshit dataset
## Use a framework like a reasonable person? (just tiny)
def build_dataset(words):
    block_size = 3 #Context lenght (numbers of characters in this case)
    X, Y = [], []
    for w in words:

        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
        #print(''.join(itos[i] for i in context), '------->', itos[ix]) #I dont understand this line of code at all
            context = context[1:] + [ix] #crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y
    
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev= build_dataset(words[n1:n2])
Xte, Yte= build_dataset(words[n2:])


In [28]:
## Cleanup With new Stuff
Xtr.shape, Ytr.shape

g = torch.Generator().manual_seed(214783647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,300), generator=g)
b1 = torch.randn((300), generator=g)
W2 = torch.randn((300,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2,]

In [29]:
sum(p.nelement() for p in parameters)

10254

In [30]:
for p in parameters:
    p.requires_grad=True

In [43]:
##fordward pass
for _ in range(1000):

    ix = torch.randint(0,Xtr.shape[0] , (32,))

    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2 # (32,27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix]) ##It's better but fuck me. 
    print(loss.item())

    
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters: 
        p. data += -.01 * p.grad

print(loss.item())

2.3932597637176514
2.6232364177703857
2.533550262451172
2.4857757091522217
2.493070363998413
2.98966121673584
2.191234588623047
2.6140406131744385
2.239069700241089
2.4716479778289795
2.530550718307495
2.538325071334839
2.2999584674835205
2.6450986862182617
2.3337562084198
2.5424787998199463
2.3672335147857666
2.336165189743042
2.6424012184143066
2.3535687923431396
2.1777946949005127
2.4242136478424072
2.5815157890319824
1.8930367231369019
2.6089601516723633
2.2055089473724365
2.386666774749756
2.751304864883423
2.2870917320251465
1.9857635498046875
2.2441976070404053
2.707542896270752
2.3263967037200928
2.6296637058258057
2.325228691101074
2.387777805328369
2.4089391231536865
2.2642557621002197
2.5582427978515625
2.729074239730835
2.470895767211914
2.177250862121582
2.358252763748169
2.4868247509002686
2.382512092590332
2.288053035736084
2.5462915897369385
2.3455586433410645
2.2572021484375
2.238403081893921
2.6446447372436523
2.350318431854248
2.500920295715332
2.777935028076172
2.58

In [32]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2 # (32,27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Ydev) ##It's better but fuck me. 
print(loss.item())

3.218350410461426


In [33]:
## Rerun with Separate dstuff


In [55]:
##Output an output. 
g = torch.Generator().manual_seed(214783647 +13)
for _ in range(25):
    out = []
    context = [0] * block_size
    while True: 
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i]for i in out))


pha.
jaxyn.
leam.
hof.
kar.
laka.
erena.
zoda.
mere.
azane.
amho.
chanili.
lar.
krkimeqdetiuecta.
kam.
nahlina.
alriaha.
san.
jaen.
twaney.
adlyyce.
aen.
don.
alon.
yayn.
